In [1]:
import requests
import json
import pandas as pd
import csv
import psycopg2

In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
import os

In [4]:
def configure():
    load_dotenv()

# Extraction Layer

In [5]:
configure()
url = os.getenv('url')

import requests

url = "https://api.rentcast.io/v1/properties/random?limit=100000"

headers = {
    "accept": "application/json",
    "X-Api-Key": os.getenv('X-Api-Key')
    
}

response = requests.get(url, headers=headers)


data = response.json()


# Save the data to a file
filename = "PropertyRecords.json"
with open(filename, "w") as file:
    json.dump(data, file, indent=4)

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [7]:
# read into a dataframe

ProjectRecords_pd = pd.read_json('PropertyRecords.json')

In [8]:
ProjectRecords_pd.drop(columns=["history","hoa"], inplace=True)

In [9]:
ProjectRecords_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                500 non-null    object 
 1   formattedAddress  500 non-null    object 
 2   addressLine1      500 non-null    object 
 3   addressLine2      109 non-null    object 
 4   city              500 non-null    object 
 5   state             500 non-null    object 
 6   zipCode           500 non-null    int64  
 7   county            499 non-null    object 
 8   latitude          500 non-null    float64
 9   longitude         500 non-null    float64
 10  propertyType      409 non-null    object 
 11  bathrooms         366 non-null    float64
 12  squareFootage     380 non-null    float64
 13  lotSize           344 non-null    float64
 14  yearBuilt         362 non-null    float64
 15  assessorID        353 non-null    object 
 16  legalDescription  349 non-null    object 
 1

In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Transformation Layer

In [11]:
#Coverting dictionary columns into string
def convert_dict_columns_to_string(ProjectRecords_pd: pd.DataFrame) -> pd.DataFrame:
    """
    Converts all dictionary-type columns in a DataFrame to string format.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: A DataFrame with dictionary columns converted to strings.
    """
    for col in ProjectRecords_pd.columns:
        # Check if the column contains dictionary entries
        if ProjectRecords_pd[col].apply(lambda x: isinstance(x, dict)).any():
            ProjectRecords_pd[col] = ProjectRecords_pd[col].apply(lambda x: str(x) if isinstance(x, dict) else x)
    return ProjectRecords_pd
ProjectRecords_pd = convert_dict_columns_to_string(ProjectRecords_pd)

In [12]:
# Replace Nan Values with appropraite defaults or remove row/columns as neccesary
ProjectRecords_pd.fillna({
    "assessorID": "Unknown",
    "legalDescription": "Not available",
    "squareFootage": 0,
    "subdivision": "Not available",
    "yearBuilt": 0,
    "bathrooms": 0,
    "lotSize": 0,
    "propertyType": "Unknown",
    "lastSalePrice":0,
    "lastSaleDate": "Not available",
    "features": "None",
    "taxAssessment": "Not available",
    "owner": "Unknown",
    "propertyTaxes": "Not available",
    "bedrooms": 0,
    "ownerOccupied": 0,
    "zoning": "Unknown",
    "addressLine2": "Not available",
    "formattedAddress": "Not Available",
    "county": "Not available"
    
}, inplace=True)

In [13]:
# Converting year built colummn from float to Integer
ProjectRecords_pd['yearBuilt'] = ProjectRecords_pd['yearBuilt'].astype(int)
ProjectRecords_pd['lastSaleDate'] = pd.to_datetime(ProjectRecords_pd['lastSaleDate'], errors='coerce')

In [14]:
# Create location Dimension
location_dim = ProjectRecords_pd[["city", "county", "state"]].drop_duplicates()
location_dim = location_dim.reset_index(drop=True).reset_index().rename(columns={"index": "location_id"})
location_dim["location_id"] += 1  # Start index from 1 instead of 0
location_dim.head(10)

,location_id,city,county,state
0,1,Detroit,Wayne,MI
1,2,Riverside,Riverside,CA
2,3,San Francisco,San Francisco,CA
3,4,Seminole,Gaines,TX
4,5,Garden City,Finney,KS
5,6,Elk Grove,Sacramento,CA
6,7,Irmo,Richland,SC
7,8,East Orange,Essex,NJ
8,9,Washington,District Of Columbia,DC
9,10,Desoto,Dallas,TX


In [15]:
# Create Property Features Dimension
description_dim = ProjectRecords_pd[["propertyType", "bathrooms", "bedrooms", "yearBuilt", "squareFootage", "lotSize"]].drop_duplicates().reset_index(drop=True)
description_dim  = description_dim.reset_index().rename(columns={"index": "description_id"})
description_dim ["description_id"] += 1  # Start index from 1 instead of 0


In [16]:
description_dim.head(30)

,description_id,propertyType,bathrooms,bedrooms,yearBuilt,squareFootage,lotSize
0,1,Single Family,1.0,0.0,1961,862.0,3485.0
1,2,Single Family,1.0,3.0,1958,1500.0,7405.0
2,3,Single Family,4.0,2.0,1948,2570.0,3584.0
3,4,Single Family,3.0,4.0,2003,2478.0,34412.0
4,5,Single Family,2.0,3.0,2019,1412.0,10086.0
5,6,Single Family,2.0,3.0,2002,2254.0,5605.0
6,7,Single Family,2.0,3.0,1997,2639.0,13939.0
7,8,Single Family,0.0,0.0,1930,2488.0,3999.0
8,9,Unknown,0.0,0.0,0,0.0,0.0
9,10,Single Family,3.5,4.0,2007,3165.0,10019.0


In [17]:
# Create time Dimension
time_dim = ProjectRecords_pd[["lastSaleDate"]].drop_duplicates().reset_index(drop=True)
time_dim = time_dim.reset_index().rename(columns={"index": "time_id"})
time_dim["time_id"] += 1  # Start index from 1 instead of 0
time_dim['year'] = time_dim['lastSaleDate'].dt.isocalendar().year  # year
time_dim['week'] = time_dim['lastSaleDate'].dt.isocalendar().week  # ISO week number
time_dim['month'] = time_dim['lastSaleDate'].dt.month_name()            # Month number
time_dim['quarter'] = time_dim['lastSaleDate'].dt.quarter          # Quarter (1-4)
time_dim['day_of_week'] = time_dim['lastSaleDate'].dt.day_name()   # Full weekday name


In [18]:
time_dim.head()

,time_id,lastSaleDate,year,week,month,quarter,day_of_week
0,1,2007-07-16 00:00:00+00:00,2007,29,July,3.0,Monday
1,2,NaT,<NA>,<NA>,NaN,NaN,NaN
2,3,2012-11-30 00:00:00+00:00,2012,48,November,4.0,Friday
3,4,2022-02-07 00:00:00+00:00,2022,6,February,1.0,Monday
4,5,2023-03-27 00:00:00+00:00,2023,13,March,1.0,Monday


In [19]:
# Create the fact table and linking of dimension table primary key with fact table
fact_columns = ["addressLine1", "city", "lastSaleDate", "propertyType", "bathrooms", "bedrooms", "yearBuilt", "squareFootage", "lotSize", 'lastSalePrice']
fact_table = ProjectRecords_pd[fact_columns]
fact_table = fact_table.merge(time_dim[["lastSaleDate", "time_id"]], on="lastSaleDate", how="left")
fact_table = fact_table.merge(location_dim[["city", "location_id"]], on="city", how="left")
fact_table = fact_table.merge(description_dim, on=["propertyType", "bathrooms", "bedrooms", "yearBuilt", "squareFootage", "lotSize"], how="left")

fact_table.drop(columns=["lastSaleDate","city","propertyType", "bathrooms", "bedrooms", "yearBuilt", "squareFootage", "lotSize"], inplace=True)


In [20]:
fact_table.tail(40)

,addressLine1,lastSalePrice,time_id,location_id,description_id
521,10801 Sw Dover Ct,670000.0,279,172,382
522,1102 Hill St,398000.0,280,94,383
523,5713 Friendswood Dr,120000.0,281,394,384
524,5920 Monticello Rd,0.0,2,395,385
525,2298 E 95th St,92000.0,282,176,386
526,2356 Beach Way SW,0.0,2,203,387
527,2319 Buttercup Cir,385000.0,283,396,388
528,4654 N Chestnut Ave,0.0,2,397,9
529,91 Park St,175000.0,284,398,389
530,846 Reading Rd,0.0,2,399,9


In [21]:
# Saving fact and dimensions table in csv format
location_dim.to_csv("location_dimension.csv", index=False),
description_dim.to_csv("description_dimension.csv", index=False),
time_dim.to_csv("time_dimension.csv", index=False),
fact_table.to_csv("property_fact.csv", index=False)

# Loading Layer

In [22]:
# develop a function to connect to pgadmin

def get_db_connection():
    load_dotenv()
    connection = psycopg2.connect(
        host=os.getenv('postgres_host'),
        database=os.getenv('postgres_database'),
        user=os.getenv('postgres_user'),
        password=os.getenv('postgres_password')
    )
    return connection

conn = get_db_connection()



In [23]:
time_dim['year'].min()

1922

In [24]:
# Creating tables
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_table_query =  '''CREATE SCHEMA IF NOT EXISTS zapbank;
    
                             DROP TABLE IF EXISTS zapbank.fact_table;
                             DROP TABLE IF EXISTS zapbank.location_dim;
                             DROP TABLE IF EXISTS zapbank.sales_dim;
                             DROP TABLE IF EXISTS zapbank.dates_dim;
                             DROP TABLE IF EXISTS zapbank.description_dim;
                             
                             CREATE TABLE zapbank.location_dim(
                                 location_id SERIAL PRIMARY KEY,
                                 city VARCHAR(100),
                                 county VARCHAR(50),
                                 state VARCHAR(50)
                             );
                             
                             CREATE TABLE zapbank.description_dim(
                                 description_id SERIAL PRIMARY KEY,
                                 propertyType VARCHAR(50),
                                 bathrooms FLOAT,
                                 bedrooms FLOAT,
                                 yearBuilt INTEGER, 
                                 squareFootage FLOAT,
                                 lotSize FLOAT
                             ); 
                             
                             CREATE TABLE zapbank.dates_dim(
                                 time_id SERIAL PRIMARY KEY,
                                 lastSaleDate DATE,
                                 year INTEGER,
                                 week INTEGER,
                                 month VARCHAR(50),
                                 quater FLOAT,
                                 day_of_week VARCHAR(50)   
                             );
                             
                             CREATE TABLE zapbank.fact_table(
                                 addressLine1 VARCHAR(255),
                                 lastSalePrice FLOAT,
                                 time_id INTEGER,
                                 location_id INTEGER,
                                 description_id INTEGER,
                                 FOREIGN KEY (time_id) REFERENCES zapbank.dates_dim(time_id),
                                 FOREIGN KEY (description_id) REFERENCES zapbank.description_dim(description_id),
                                 FOREIGN KEY (location_id) REFERENCES zapbank.location_dim(location_id)
                             );
                             
                             '''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()
    
create_tables()

In [25]:
# create a function to load the csv data into the database
def load_data_from_csv_to_table(csv_path,  table_name):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r', encoding="utf-8") as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                placeholders = ', '.join(['%s'] * len(row))
                query = f'INSERT INTO {table_name} VALUES({placeholders});'
                cursor.execute(query, row)
    conn.commit()
    cursor.close()
    conn.close()     

In [26]:
# location dimension table
location_csv_path = r'/Users/OYETAYOADEDAMOLA/Documents/AmdariProject/Project1/location_dimension.csv'
load_data_from_csv_to_table(location_csv_path,'zapbank.location_dim')

In [27]:
# description dimension table
description_csv_path = r'/Users/OYETAYOADEDAMOLA/Documents/AmdariProject/Project1/description_dimension.csv'
load_data_from_csv_to_table(description_csv_path,'zapbank.description_dim')

In [28]:
# create a function to load the csv data into the database (specifically for sales)
def load_data_from_csv_to_sales_table(csv_path, table_name):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r', encoding="utf-8") as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                # convert empty strings (or 'Not available') in date column to None(Null in SQL)
                row = [None if (cell == "" or cell == "Not available") else cell for cell in row]
                placeholders = ', '.join(['%s'] * len(row))
                query = f'INSERT INTO {table_name} VALUES({placeholders});'
                cursor.execute(query, row)
    conn.commit()
    cursor.close()
    conn.close()


In [29]:
# time dimension table
time_csv_path = r'/Users/OYETAYOADEDAMOLA/Documents/AmdariProject/Project1/time_dimension.csv'
load_data_from_csv_to_sales_table(time_csv_path,'zapbank.dates_dim')

In [30]:
# fact table
fact_csv_path = r'/Users/OYETAYOADEDAMOLA/Documents/AmdariProject/Project1/property_fact.csv'
load_data_from_csv_to_table(fact_csv_path,'zapbank.fact_table')

In [31]:
print('All Data has been loaded successfully into their respective schema and tables')

All Data has been loaded successfully into their respective schema and tables
